# Function

In [1]:
from pathlib import Path
import subprocess
import pickle
import numpy as np
import pandas as pd
import time
import multiprocessing
import concurrent.futures
import os
import jax
import jax.numpy as jnp
from evosax import BIPOP_CMA_ES
from datetime import datetime
import shutil


In [2]:
def make_log_weight(dim):
    log_weight = np.log(dim + 1) - np.log(np.arange(1, dim + 1))
    return log_weight    

In [3]:
def random_choose_index(n):
    global df_indexes, debug_mode, df, lose_instanse_percent

    first_win_idx = np.where(df.gain_percent >= 0)[0][0] # if first_win_idx can't construct, we lose completely
    # left_weight = make_log_weight(first_win_idx + 1)
    # right_weight = np.full(len(df_indexes) - first_win_idx - 1, left_weight[-1])
    left_weight = np.full(first_win_idx, lose_instanse_percent)
    right_weight = np.full(len(df_indexes) - first_win_idx, 1 - lose_instanse_percent)
    prob = np.hstack((left_weight, right_weight))
    prob /= prob.sum()
    if debug_mode:
        print("first_win_idx", first_win_idx)
        print("len(left_weight)", len(left_weight))
        print("len(right_weight)", len(right_weight))
        print("prob[:first_win_idx]")
        print(prob[:first_win_idx])
        print("prob[first_win_idx:]")
        print(prob[first_win_idx:])

    return np.random.choice(df_indexes, n, p=prob, replace=False)

In [4]:
def run_command(command):
    result = subprocess.run(command, capture_output=True)
    assert (
        result.returncode == 0
    ), f"""
command:
{' '.join(command)}
returncode: {result.returncode}
stderr:
{result.stderr.decode()}
stdout:
{result.stdout.decode()}
"""
    return result

In [5]:
def run_a_instance(chain_flags, instance_name, acopp_profit, seed):
    global debug_mode, debug_time, acopp_dir, sol_dir

    command = [
        'python3',
        f'{acopp_dir}/run.py',
        '--acopp_dir',
        str(acopp_dir),
        '--instance_name',
        instance_name,
        '--run_only',
        '--experiment',
        # '--no_log',
        '--sol_dir',
        str(sol_dir),
        '--silent',
        '1',
        "--postfix",
        str(time.time()),
        "--random_seed",
        str(seed),
        
        "--chain_flags",
        str(chain_flags),
    ]

    if debug_mode:
        command += ["--time", str(debug_time)]
    
    result = run_command(command)
    stdout_log = result.stdout.decode()
    profit = int(stdout_log)
    
    gain_percent = (profit - acopp_profit) / acopp_profit * 100
    return gain_percent

In [6]:
def to_arr_flag(a_list):
    global n_decimal
    if n_decimal is not None:
        arr_flag = map(lambda x : f"{x:.{n_decimal}f}", a_list)
    else:
        arr_flag = map(str, a_list)
    arr_flag = ":".join(arr_flag)
    return arr_flag

In [7]:
def make_init_min_max_values(name, params):
    global index_dict, n_decimal
    min_min_value = globals()["min_min_" + name]
    max_max_value = globals()["max_max_" + name]
    
    init_value = params[index_dict[name]]
    left_value = params[index_dict[f"left_{name}"]]
    _mid_value = params[index_dict[f"_mid_{name}"]]
    right_value = params[index_dict[f"right_{name}"]]
    
    sum_value = left_value + _mid_value + right_value
    left_value = left_value / sum_value * (max_max_value - min_min_value)
    _mid_value = _mid_value / sum_value * (max_max_value - min_min_value)
    min_value = min_min_value + left_value
    max_value = min_value + _mid_value
    
    if n_decimal is not None:
        min_value = np.round(min_value, n_decimal)
        max_value = np.round(max_value, n_decimal)

    return init_value, min_value, max_value

In [8]:
def to_chain_flags(params):
    global index_dict, max_max_rho, min_min_rho, max_max_indv_ants, min_min_indv_ants
    global mean_order, std_order, rho_order, indv_ants_order

    init_indv_ants, min_indv_ants, max_indv_ants = make_init_min_max_values("indv_ants", params)
    indv_ants_list = []
    for x in indv_ants_order:
        indv_ants_list.append(locals()[x])
    indv_ants_arr = to_arr_flag(indv_ants_list)

    pop_size = params[index_dict["pop_size"]]
    mean_arr = to_arr_flag([params[index_dict[x]] for x in mean_order])
    std_arr = to_arr_flag([params[index_dict[x]] for x in std_order])
    rho_arr = to_arr_flag([params[index_dict["init_rho"]]])

    chain_flags = f"--adapt_evap --cmaes --lambda {pop_size} --mean_ary {mean_arr} --std_ary {std_arr} --adpt_rho {rho_arr} --indv_ants {indv_ants_arr}"
    return chain_flags

In [9]:
def make_priority_order(row):
    global min_gain
    if row.gain_percent < 0:
        return row.gain_percent - min_gain + 1
    else:
        return - row.gain_percent
    

def sort_by_priority(df):
    global min_gain
    min_gain = df.gain_percent.min()

    df["priority_order"] = df.apply(make_priority_order, axis=1)
    df.sort_values(by="priority_order", ascending=False, inplace=True)
    df.drop(columns=["priority_order"], inplace=True)


def update_df(picked_idxes, gain_percents):
    assert len(picked_idxes) == len(gain_percents)

    global df

    for i, df_idx in enumerate(picked_idxes):
        # df.loc[df_idx, "gain_percent"] = gain_percents[i]
        df.loc[df_idx, "gain_percent"] = (df.loc[df_idx, "gain_percent"] + gain_percents[i]) / 2
        # df.loc[df_idx, "gain_percent"] = round(df.loc[df_idx, "gain_percent"], 3)
    # sort_by_priority(df)
    df.sort_values(by="gain_percent", inplace=True)


In [10]:
def evaluate_pop(pop_params):
    global n_run_each_trail, executor, eval_call_count, df
    global prev_prev_best, prev_best

    tasks = []
    chain_flagss = []
    for params in pop_params:
        chain_flagss.append(to_chain_flags(params))
    picked_idxes = random_choose_index(n_run_each_trail)

    for df_idx in picked_idxes:
        for _chain_flags in chain_flagss:
            tasks.append((
                _chain_flags,
                df.loc[df_idx].instance,
                df.loc[df_idx].acopp_profit,
                eval_call_count+1,
                ))
            eval_call_count += 1

    gain_percents = executor.map(run_a_instance, *zip(*tasks))
    gain_percents = np.array(list(gain_percents))
    gain_percents = gain_percents.reshape((n_run_each_trail, len(pop_params)))
    

    # objective_values = gain_percents
    # objective_values[objective_values > 0] = 0
    # objective_values = objective_values.T.mean(axis=1)

    objective_values = gain_percents.T.mean(axis=1)
    fitness = jnp.array(- objective_values)     # minimize
    best_idx = fitness.argmin()                 # minimize
    
    prev_prev_best = prev_best
    prev_best = pop_params[best_idx]
    update_df(picked_idxes, gain_percents[:, best_idx])
    # update_df(picked_idxes, gain_percents.mean(axis=1))
    
    return fitness

In [11]:
def save_study():
    global state, rng, strategy, es_params
    global eval_call_count, index_dict
    global save_path, backup_dir
    global prev_best, prev_prev_best
    global df, csv_path
    global min_min_rho, max_max_rho

    study = {
        "strategy": strategy,
        "es_params": es_params,
        "state": state,
        "rng": rng,

        "index_dict": index_dict,
        
        "prev_prev_best": prev_prev_best,
        "prev_best": prev_best,
        "eval_call_count": eval_call_count,

        "min_min_rho": min_min_rho,
        "max_max_rho": max_max_rho,
    }
    if prev_best is not None:
        study["best_chain_flags"] = to_chain_flags(prev_best)
    if prev_prev_best is not None:
        study["second_best_chain_flags"] = to_chain_flags(prev_prev_best)

    with open(save_path, "wb") as f:
        pickle.dump(study, f)

    df.to_csv(csv_path, index=False)

    # Backup
    _now = time.time()
    df.to_csv(backup_dir / f"{_now}.csv", index=False)
    with open(backup_dir / f"{_now}.pkl", "wb") as f:
        pickle.dump(study, f)


In [12]:
def load_study():
    global state, rng, strategy, es_params
    global eval_call_count, index_dict
    global save_path
    global prev_best, prev_prev_best
    global df, csv_path
    global min_min_rho, max_max_rho

    with open(save_path, "rb") as f:
        study = pickle.load(f)
    
    state = study["state"]
    rng = study["rng"]
    es_params = study["es_params"]
    strategy = study["strategy"]
    
    index_dict = study["index_dict"]

    prev_best = study["prev_best"]
    prev_prev_best = study["prev_prev_best"]
    eval_call_count = study["eval_call_count"]

    min_min_rho = study["min_min_rho"]
    max_max_rho = study["max_max_rho"]

    df = pd.read_csv(csv_path)


In [13]:
def modify_pop(pop_params):
    global index_dict, eval_call_count, prev_best, prev_prev_best, n_decimal

    pop_params = np.array(pop_params)
    replace_idx = np.random.choice(np.arange(len(pop_params)), 2, replace=False)
    if prev_best is not None:
        pop_params[replace_idx[0]] = prev_best
    if prev_prev_best is not None:
        pop_params[replace_idx[1]] = prev_prev_best

    pop_params[:, index_dict["indv_ants"]] = np.round(pop_params[:, index_dict["indv_ants"]])
    pop_params[:, index_dict["pop_size"]] = np.round(pop_params[:, index_dict["pop_size"]] / 2) * 2
    if n_decimal is not None:
        pop_params = np.round(pop_params, n_decimal)

    return jnp.array(pop_params)

In [14]:
def win_percent():
    global df
    return (df.gain_percent >= 0).sum() / len(df.gain_percent) * 100

In [15]:
def clean():
    global sol_dir

    if os.path.exists("./errcmaes.err"):
        os.remove("./errcmaes.err")
    if os.path.exists("./actparcmaes.par"):
        os.remove("./actparcmaes.par")
    if os.path.exists(sol_dir):
        shutil.rmtree(sol_dir)

In [16]:
def set_init(name, mean_value, min_value=-3.4e+38, max_value=3.4e+38, std_value=None, std_factor=5):
    global clip_min, clip_max, init_min, init_max, sigma_init, index_dict

    if std_value is None:
        assert min_value != -3.4e+38 and max_value != 3.4e+38
        std_value = (max_value - min_value) / std_factor

    idx = index_dict[name]
    clip_min[idx] = min_value
    clip_max[idx] = max_value
    init_min[idx] = init_max[idx] = mean_value
    sigma_init[idx] = std_value
        

In [17]:
def min_max_to_left_mid_right(min_value, max_value, min_min_value, max_max_value):
    min_value = (min_value - min_min_value) / (max_max_value - min_min_value)
    max_value = (max_value - min_min_value) / (max_max_value - min_min_value)
    left_value = min_value
    _mid_value = max_value - min_value
    right_value = 1 - max_value

    return left_value, _mid_value, right_value

In [18]:
def set_init_min_max(name, min_value, max_value):
    min_min_value = globals()["min_min_" + name]
    max_max_value = globals()["max_max_" + name]

    left_value, _mid_value, right_value = min_max_to_left_mid_right(min_value, max_value, min_min_value, max_max_value)
    set_init(
        name=f"left_{name}",
        min_value= 0,
        max_value= 1,
        mean_value= left_value,
    )
    set_init(
        name=f"_mid_{name}",
        min_value= 0,
        max_value= 1,
        mean_value= _mid_value,
    )
    set_init(
        name=f"right_{name}",
        min_value= 0,
        max_value= 1,
        mean_value= right_value,
    )

In [19]:
def set_init_with_std(
        name, mean_value, 
        min_value=-3.4e+38, max_value=3.4e+38,
        std_value=None, std_factor=5,
        min_std_value=0.0001, max_std_value=3.4e+38,
        std_std_factor=5, std_std_value=None,
        ):
    
    if std_value is None:
        assert min_value != -3.4e+38 and max_value != 3.4e+38
        std_value = (max_value - min_value) / std_factor
    set_init(
        name=f"{name}_mean",
        min_value=min_value,
        max_value=max_value,
        mean_value=mean_value,
        std_value=std_value,
        std_factor=std_factor,
        )
    set_init(
        name=f"{name}_std",
        min_value=min_std_value,
        max_value=max_std_value,
        mean_value=std_value,
        std_value=std_std_value,
        std_factor=std_std_factor,
        )
        

# Config

In [20]:
debug_mode = False
# debug_mode = True
debug_time = 5

n_jobs = max(1, multiprocessing.cpu_count() // 2)
acopp_dir = Path("../")
signal_path = Path("./signal.txt")
sol_dir = Path("/home/user2/temp/evosax_tuning/solutions")
n_decimal = None
# n_decimal = 2
# lose_instanse_percent = 0.95
lose_instanse_percent = 0.5

if not debug_mode:
    n_run_each_trail = 21
else:
    n_run_each_trail = 2
    optim_popsize = 4

if not debug_mode:
    experiment_name = "evosax_tuning"
    experiment_dir = Path("/home/user2/experiments") / experiment_name
else:
    experiment_name = "temp_evosax_tuning"
    experiment_dir = Path("./") / experiment_name

csv_path = experiment_dir / "gain_percent.csv"
save_path = experiment_dir / "study.pkl"
backup_dir = experiment_dir / "backup"

# Prepare

In [21]:
os.makedirs(experiment_dir, exist_ok=True)
os.makedirs(sol_dir, exist_ok=True)
os.makedirs(backup_dir, exist_ok=True)
os.makedirs(save_path.parent, exist_ok=True)

In [22]:
# Build
assert os.path.isdir(acopp_dir)
command = [
    'python3',
    f'{acopp_dir}/run.py',
    '--acopp_dir',
    str(acopp_dir),
    '--build_only',
    '--experiment'
    ]
result = run_command(command)
print(result.stdout.decode())

$ cmake -DCMAKE_EXPORT_COMPILE_COMMANDS:BOOL=TRUE -G Unix Makefiles -S.. -B../temp_build_experiment -DCMAKE_BUILD_TYPE:STRING=Release
-- Configuring done (0.1s)
-- Generating done (0.3s)
-- Build files have been written to: /mnt/c/home/vincent/acoplusplus_thop_modified/src/aco++/temp_build_experiment

$ cmake --build ../temp_build_experiment -j 6
[100%] Built target acothop




In [23]:
min_min_indv_ants = 2
max_max_indv_ants = 100

In [24]:
mean_order = [
    "alpha_mean",
    "beta_mean",
    "par_a_mean",
    "par_b_mean",
    "par_c_mean",
    "elite_prob_mean",
    "neighbour_prob_mean",
    "left_rho_mean",
    "_mid_rho_mean",
    "right_rho_mean",
]
std_order = [
    "alpha_std",
    "beta_std",
    "par_a_std",
    "par_b_std",
    "par_c_std",
    "elite_prob_std",
    "neighbour_prob_std",
    "left_rho_std",
    "_mid_rho_std",
    "right_rho_std",
]
rho_order = [
    "init_rho",
    # "min_rho",
    # "max_rho",
]
indv_ants_order = [
    "init_indv_ants",
    "min_indv_ants",
    "max_indv_ants",
]

In [25]:
df = pd.read_csv(csv_path)
n_instance = len(df.instance)
df_indexes = np.arange(n_instance)
assert n_instance == 432

# IMPORTANCE: One time preparation

In [26]:
# variable_list = [
#     "pop_size",
#     "init_rho",

#     "indv_ants",
#     "left_indv_ants",
#     "_mid_indv_ants",
#     "right_indv_ants",

# ] + mean_order + std_order

# index_dict = dict()
# for idx, key in enumerate(variable_list):
#     index_dict[key] = idx

# dim = len(index_dict)
# if not debug_mode:
#     optim_popsize = 4 + 3 * np.log(dim)
#     optim_popsize = np.ceil(optim_popsize / 2) * 2
#     optim_popsize = int(optim_popsize)
# rng = jax.random.PRNGKey(int(time.time()))
# strategy = BIPOP_CMA_ES(popsize=optim_popsize, num_dims=dim)
# es_params = strategy.default_params
# clip_max= np.zeros((dim,))
# clip_min= np.zeros((dim,))
# init_min= np.zeros((dim,))
# init_max= np.zeros((dim,))
# sigma_init= np.zeros((dim,))
# prev_best = None
# prev_prev_best = None
# min_min_rho = 0.0001 # must be the same as in inout.cpp
# max_max_rho = 0.9999 # must be the same as in inout.cpp
# eval_call_count = 17700
# # --adapt_evap --cmaes --lambda 16.0 --mean_ary 0.78049654:1.9156744:0.680437:0.802763:0.38080865:0.08790362:0.91120124:0.000988643:0.6528713:0.3308815 --std_ary 1e-04:3.2758715:0.06292335:0.34004503:0.24407929:0.5784137:0.06291627:0.055570904:0.14468291:0.22623919 --adpt_rho 0.65771425 --indv_ants 17.0:16.663479:57.95118
# set_init(
#     name="pop_size",
#     min_value= 4,
#     mean_value= 16,
#     std_value= 2,
# )
# set_init(
#     name="init_rho",
#     min_value= 0.0001,
#     max_value= 0.9999,
#     mean_value= 0.65771425,
#     std_value= 0.253226,
# )
# set_init(
#     name="indv_ants",
#     min_value= 2,
#     mean_value= 17,
#     std_value= 2,
# )
# set_init_min_max(
#     name="indv_ants",
#     min_value=16.663479,
#     max_value=57.95118,
# )

# set_init_with_std(
#     name="alpha",
#     min_value= 0.0001,
#     mean_value= 0.78049654,
#     std_value= 1.523180,
#     std_std_value=1,
# )
# set_init_with_std(
#     name="beta",
#     min_value= 0.0001,
#     mean_value= 1.9156744,
#     std_value= 3.2758715,
#     std_std_value= 1,
# )

# set_init_with_std(
#     name="par_a",
#     min_value= 0.0001,
#     max_value= 1,
#     mean_value= 0.5,
#     std_value= 0.2,
#     std_std_value=0.1,
# )
# set_init_with_std(
#     name="par_b",
#     min_value= 0.0001,
#     max_value= 1,
#     mean_value= 0.5,
#     std_value= 0.2,
#     std_std_value=0.1,
# )
# set_init_with_std(
#     name="par_c",
#     min_value= 0.0001,
#     max_value= 1,
#     mean_value= 0.5,
#     std_value= 0.2,
#     std_std_value=0.1,
# )
# set_init_with_std(
#     name="elite_prob",
#     min_value= 0,
#     max_value= 0.9999,
#     mean_value= 0.08790362,
#     std_value= 0.5784137,
#     std_std_value=0.1,
# )
# set_init_with_std(
#     name="neighbour_prob",
#     min_value= 0,
#     max_value= 1,
#     mean_value= 0.9,
#     std_value= 0.1,
#     std_std_value=0.1,
# )
# set_init_with_std(
#     name="left_rho",
#     min_value= 0,
#     max_value= 1,
#     mean_value= 0.01,
#     std_value= 0.055570904,
#     std_std_value= 0.1,
# )
# set_init_with_std(
#     name="_mid_rho",
#     min_value= 0,
#     max_value= 1,
#     mean_value= 0.6528713,
#     std_value= 0.14468291,
#     std_std_value= 0.1,
# )
# set_init_with_std(
#     name="right_rho",
#     min_value= 0,
#     max_value= 1,
#     mean_value= 0.3308815,
#     std_value= 0.22623919,
#     std_std_value= 0.1,
# )

# es_params = es_params.replace(
#         strategy_params=es_params.strategy_params.replace(
#             clip_min=jnp.array(clip_min),
#             clip_max=jnp.array(clip_max),
#             init_min=jnp.array(init_min),
#             init_max=jnp.array(init_max),
#             sigma_init=jnp.array(sigma_init),
#             )
#         )
# state = strategy.initialize(rng, es_params)

# assert not os.path.exists(save_path)
# assert not os.path.exists(save_path)
# assert not os.path.exists(save_path)
# save_study()

# Tuning

In [27]:
load_study()

with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
    # minimize
    while True:
        rng, rng_ask = jax.random.split(rng, 2)

        if state.restart_state.restart_next:
            print(f"--> Restarted Strategy: {eval_call_count} eval calls")
            print_update = True
        else:
            print_update = False

        x, state = strategy.ask(rng_ask, state, es_params)

        if print_update:
            print(f"--> New Popsize: {state.restart_state.active_popsize}")

        x = modify_pop(x)
        fitness = evaluate_pop(x)
        state = strategy.tell(x, fitness, state, es_params)
        
        state = state.replace(
            restart_state = state.restart_state.replace(
                restart_next = state.restart_state.restart_next.any()
            )
        )

        clean()
        save_study()

        print(f"{datetime.now()} | eval_call_count: {eval_call_count} | mean_gain_percent: {df.gain_percent.mean():.4f} | win_percent: {win_percent():.4f} | best_chain_flags: {to_chain_flags(prev_best)}")
        with open(signal_path, "rt") as f:
            if f.read() == "BREAK":
                break

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


2023-08-07 18:56:15.797374 | eval_call_count: 28186 | mean_gain_percent: -0.6070 | win_percent: 30.3241 | best_chain_flags: --adapt_evap --cmaes --lambda 18.0 --mean_ary 0.93917453:2.324807:0.27753335:0.32051715:0.3800142:0.6131718:0.9173933:0.07920453:0.79138726:0.6168047 --std_ary 1e-04:1.8921341:0.30154198:0.0934514:0.29689157:0.68888366:0.14120248:0.21735258:0.23919487:0.30166396 --adpt_rho 0.6962349 --indv_ants 11.0:20.88983:67.13563
2023-08-07 20:14:07.411867 | eval_call_count: 28480 | mean_gain_percent: -0.5902 | win_percent: 30.7870 | best_chain_flags: --adapt_evap --cmaes --lambda 18.0 --mean_ary 0.8225838:3.3035645:0.29395422:0.37495235:0.4243315:0.6926936:0.9007977:0.0753745:0.77331:0.6399943 --std_ary 1e-04:2.1483555:0.29017612:0.11110502:0.31678465:0.67778057:0.11383935:0.2357051:0.24624643:0.32316136 --adpt_rho 0.66205674 --indv_ants 10.0:24.021074:66.10524
2023-08-07 23:17:22.924412 | eval_call_count: 28774 | mean_gain_percent: -0.5736 | win_percent: 30.7870 | best_chain